In [ ]:
%pip install --upgrade git+https://github.com/huggingface/transformers.git accelerate

In [ ]:
import torch
print(torch.version.cuda)

## Set up the pipeline with openai/whisper-large-v3

In [4]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=False, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
import os
import json

def transcribe_and_save(audio_file_path, output_folder):
    # Process the audio file using the pipe function
    result = pipe(audio_file_path, return_timestamps=True)
    # Extract the base name without the extension and create new file names
    base_name = os.path.basename(os.path.splitext(audio_file_path)[0])
    text_file_name = os.path.join(output_folder, f"{base_name}.txt")
    json_file_name = os.path.join(output_folder, f"{base_name}.json")

    # Write the transcription to a text file with timestamps
    with open(text_file_name, 'w', encoding='utf-8') as text_file:
        for chunk in result["chunks"]:
            # Formatting the timestamp
            start, end = chunk['timestamp']
            timestamp = f"({start}, {end})"
            text_file.write(f"[{timestamp}]: {chunk['text']}\n")

    # Save the transcription as a JSON file
    with open(json_file_name, 'w', encoding='utf-8') as json_file:
        json.dump(result["chunks"], json_file, indent=4)

    print(f"Transcription saved to {text_file_name}")
    print(f"Transcription JSON saved to {json_file_name}")

def process_folder(input_folder, output_folder):
    for file in os.listdir(input_folder):
        if file.endswith(".mp3"):
            audio_file_path = os.path.join(input_folder, file)
            transcribe_and_save(audio_file_path, output_folder)
            print(f"Processed {file}")

# Usage: specify the folder containing the MP3 files and the output folder for transcriptions
process_folder('./Audio', './Transcript')



c:\Users\dougl\miniconda3\envs\asr\Lib\site-packages\transformers\models\whisper\modeling_whisper.py:694: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
c:\Users\dougl\miniconda3\envs\asr\Lib\site-packages\torch\nn\modules\conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\cb\pytorch_100000

Transcription saved to ./Transcript\Developer’s Guide to Customizing Microsoft Copilot - Micrososft Build 2024.txt
Transcription JSON saved to ./Transcript\Developer’s Guide to Customizing Microsoft Copilot - Micrososft Build 2024.json
Processed Developer’s Guide to Customizing Microsoft Copilot - Micrososft Build 2024.mp3
